In [1]:
# TA-Lib 설치 코드
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h516909a_0.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py37ha21ca33_2.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.7/dist-packages/ lib/python3.7/site-packages/talib --strip-components=3
import talib

# 야후 파이낸스설치
!pip install yfinance
import yfinance as yf

import pandas as pd
import numpy as np

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3733    0  3733    0     0  16372      0 --:--:-- --:--:-- --:--:-- 16301
100  503k  100  503k    0     0   826k      0 --:--:-- --:--:-- --:--:--  826k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3741    0  3741    0     0  18428      0 --:--:-- --:--:-- --:--:-- 18428
100  406k  100  406k    0     0   727k      0 --:--:-- --:--:-- --:--:-- 1392k
     |████████████████████████████████| 63 kB 622 kB/s 
     |████████████████████████████████| 6.4 MB 10.0 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling l

In [2]:
# 네이버 데이터 불러오기
naver_df= df = yf.download('035720.KS',start = '2005-01-01') # 원하는 주식의 종목코드+.KS
sox_df = yf.download('^SOX', start = '2000-01-01')
vix_df = yf.download('^VIX', start = '2000-01-01')
snp500_df = yf.download('^GSPC', start = '2000-01-01')
kospi_df= yf.download('^KS11', start = '2000-01-01')

kospi_df.to_csv('kospi.csv')
naver_df.to_csv('naver.csv')
sox_df.to_csv('sox_df.csv')
vix_df.to_csv('vix_df.csv')
snp500_df.to_csv('s&p500.csv')

naver_df = pd.read_csv('naver.csv',index_col='Date', parse_dates=True)
sox_df = pd.read_csv('sox_df.csv', index_col = 'Date', parse_dates = True)
vix_df = pd.read_csv('vix_df.csv', index_col = 'Date', parse_dates = True)
snp500_df = pd.read_csv('s&p500.csv', index_col = 'Date', parse_dates = True)
kospi_df = pd.read_csv('kospi.csv', index_col = 'Date', parse_dates = True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [3]:
snp500_df = snp500_df.loc[:,['Close']].copy()
snp500_df.rename(columns={'Close':'S&P500'},inplace=True)
sox_df = sox_df.loc[:,['Close']].copy()
sox_df.rename(columns={'Close':'SOX'},inplace=True)
vix_df = vix_df.loc[:,['Close']].copy()
vix_df.rename(columns={'Close':'VIX'},inplace=True)


df = df.join(snp500_df,how='left')
df = df.join(sox_df,how='left')
df = df.join(vix_df,how='left')

In [4]:
kospi_df['KOSPI'] = kospi_df['Close'].copy()

In [5]:
kospi_df['kosATR'] = talib.ATR(kospi_df['High'], kospi_df['Low'], kospi_df['Close'], timeperiod=14)

kospi_df['kosADX14'] = talib.ADXR(kospi_df['High'], kospi_df['Low'], kospi_df['Close'], timeperiod=14)


kospi_df = kospi_df.drop(['Open','High','Low','Close','Adj Close','Volume'],axis=1)
df = df.join(kospi_df, how = 'left')

In [6]:
df

,Open,High,Low,Close,Adj Close,Volume,S&P500,SOX,VIX,KOSPI,kosATR,kosADX14
Date,,,,,,,,,,,,
2005-01-03,4700.0,4770.0,4650.0,4750.0,4482.411621,3150000,1202.079956,424.260010,14.080000,893.710022,12.755826,13.114406
2005-01-04,4760.0,4790.0,4580.0,4650.0,4388.044922,2352500,1188.050049,410.359985,13.980000,886.900024,12.591840,12.609550
2005-01-05,4600.0,4830.0,4550.0,4780.0,4510.721680,3658000,1183.739990,404.250000,14.090000,885.190002,12.672425,12.186528
2005-01-06,4820.0,5340.0,4720.0,5080.0,4793.821289,11569000,1187.890015,402.140015,13.580000,871.280029,12.840821,11.941418
2005-01-07,5080.0,5430.0,4980.0,5060.0,4774.948242,9490500,1186.189941,407.559998,13.490000,870.840027,12.792908,12.010606
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-04,85100.0,87500.0,84500.0,87100.0,87100.000000,1870626,4500.529785,3453.070068,23.219999,2750.260010,50.192953,26.325412
2022-02-07,87300.0,88000.0,85000.0,88000.0,88000.000000,2224116,4483.870117,3452.330078,22.860001,2745.060059,48.876314,26.825926
2022-02-08,88000.0,88000.0,88000.0,88000.0,88000.000000,0,4521.540039,3535.129883,21.440001,2746.469971,49.180158,27.281226


In [7]:
# 기술지표 가공
df['log_return'] = np.log(1 + df['Adj Close'].pct_change())
df['CCI'] = talib.CCI(df['High'], df['Low'], df['Adj Close'], timeperiod=14)


#1.RA : Standard deviation rolling average
# Moving Average
df['MA10'] = talib.SMA(df['Close'],timeperiod=10)
df['MA60'] = talib.SMA(df['Close'],timeperiod=60)

df['MA60_volumne'] = talib.SMA(df['Volume'], timeperiod = 60)
df['RASD5'] = talib.SMA(talib.STDDEV(df['Close'], timeperiod=5, nbdev=1),timeperiod=5)
df['RASD10'] = talib.SMA(talib.STDDEV(df['Close'], timeperiod=5, nbdev=1),timeperiod=10)

#2.MACD : Moving Average Convergence/Divergence
macd, macdsignal, macdhist = talib.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
df['MACD'] = macd 

# Momentum Indicators
#3.CCI : Commodity Channel Index

df['CCI60'] = talib.CCI(df['High'], df['Low'], df['Close'], timeperiod=60)
df['CCI60'] = talib.CCI(df['High'], df['Low'], df['Close'], timeperiod=90)
# Volatility Indicators 

#4.ATR : Average True Range
df['ATR14'] = talib.ATR(df['High'], df['Low'], df['Close'], timeperiod=14)

#7.MTM
df['MTM3'] = talib.MOM(df['Close'], timeperiod=3)

#8.ROC : Rate of change : ((price/prevPrice)-1)*100
df['ROC60'] = talib.ROC(df['Close'], timeperiod=60)
df['ROC90'] = talib.ROC(df['Close'], timeperiod=30)

#9.WPR : william percent range (Williams' %R)
df['WPR14'] = talib.WILLR(df['High'], df['Low'], df['Close'], timeperiod=14)

#10.ADXR
df['ADXR14'] = talib.ADXR(df['High'], df['Low'], df['Close'], timeperiod=14)

#11.ADX
df['ADX14'] = talib.ADX(df['High'], df['Low'], df['Close'], timeperiod=14)

In [8]:
df.head()

,Open,High,Low,Close,Adj Close,Volume,S&P500,SOX,VIX,KOSPI,kosATR,kosADX14,log_return,CCI,MA10,MA60,MA60_volumne,RASD5,RASD10,MACD,CCI60,ATR14,MTM3,ROC60,ROC90,WPR14,ADXR14,ADX14
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2005-01-03,4700.0,4770.0,4650.0,4750.0,4482.411621,3150000,1202.079956,424.260010,14.08,893.710022,12.755826,13.114406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-04,4760.0,4790.0,4580.0,4650.0,4388.044922,2352500,1188.050049,410.359985,13.98,886.900024,12.591840,12.609550,-0.021277,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-05,4600.0,4830.0,4550.0,4780.0,4510.721680,3658000,1183.739990,404.250000,14.09,885.190002,12.672425,12.186528,0.027573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-06,4820.0,5340.0,4720.0,5080.0,4793.821289,11569000,1187.890015,402.140015,13.58,871.280029,12.840821,11.941418,0.060871,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,330.0,NaN,NaN,NaN,NaN,NaN
2005-01-07,5080.0,5430.0,4980.0,5060.0,4774.948242,9490500,1186.189941,407.559998,13.49,870.840027,12.792908,12.010606,-0.003945,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,410.0,NaN,NaN,NaN,NaN,NaN


In [9]:
df.to_csv('stock_data.csv')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4249 entries, 2005-01-03 to 2022-02-10
Data columns (total 28 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          4249 non-null   float64
 1   High          4249 non-null   float64
 2   Low           4249 non-null   float64
 3   Close         4249 non-null   float64
 4   Adj Close     4249 non-null   float64
 5   Volume        4249 non-null   int64  
 6   S&P500        4119 non-null   float64
 7   SOX           4119 non-null   float64
 8   VIX           4119 non-null   float64
 9   KOSPI         4226 non-null   float64
 10  kosATR        4226 non-null   float64
 11  kosADX14      4226 non-null   float64
 12  log_return    4248 non-null   float64
 13  CCI           4236 non-null   float64
 14  MA10          4240 non-null   float64
 15  MA60          4190 non-null   float64
 16  MA60_volumne  4190 non-null   float64
 17  RASD5         4241 non-null   float64
 18  RASD10    